In [ ]:
!pip install bert_score
!pip install transformers datasets evaluate sentencepiece
!pip install datasets
!pip install ipywidgets
!pip install evaluate
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

train_dataset = load_dataset("wmt16", "de-en", split="train[:50000]")
val_dataset = load_dataset("wmt16", "de-en", split="validation")
test_dataset = load_dataset("wmt16", "de-en", split="test")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from evaluate import load

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

def translate(text):
    model_input = tokenizer("translate English to German: " + text, return_tensors="pt", padding=True)
    translated_tokens = model.generate(**model_input, max_length=512, num_beams=4)
    return tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

bleu = load("bleu")
meteor = load("meteor")
bertscore = load("bertscore", lang="de")

def evaluate_dataset(dataset):
    references = []
    predictions = []
    for i in range(len(dataset)):
        ex = dataset[i]

        english_text = ex['translation']['en']
        german_text = ex['translation']['de']

        translated_text = translate(english_text)

        predictions.append(translated_text)
        references.append([german_text])

    bleu_score = bleu.compute(predictions=predictions, references=references)
    meteor_score = meteor.compute(predictions=predictions, references=references)
    bertscore_result = bertscore.compute(predictions=predictions, references=references, lang="de")

    return bleu_score, meteor_score, bertscore_result

bleu_score, meteor_score, bertscore_result = evaluate_dataset(val_dataset)

print("BLEU Score:", bleu_score)
print("METEOR Score:", meteor_score)
print("BERTScore:", bertscore_result)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BLEU Score: {'bleu': 0.2812344185106677, 'precisions': [0.5926729385722191, 0.33634841650002323, 0.2160871694417238, 0.14522467552613888], 'brevity_penalty': 1.0, 'length_ratio': 1.0206732941708088, 'translation_length': 45175, 'reference_length': 44260}
METEOR Score: {'meteor': 0.552899836584454}
BERTScore: {'precision': [0.922523558139801, 0.8907819390296936, 0.9487404823303223, 0.8868699073791504, 0.9417358636856079, 0.853186845779419, 0.9534710645675659, 0.868914008140564, 0.8954386711120605, 0.9476878046989441, 0.7796664834022522, 0.9396441578865051, 0.9311310052871704, 0.8699208498001099, 0.8598196506500244, 0.8581067323684692, 0.9577564001083374, 0.7675561904907227, 0.8691654205322266, 0.9007344841957092, 0.8464643359184265, 0.9132484197616577, 0.8719235062599182, 0.7746807336807251, 0.8242511749267578, 0.8306722640991211, 0.9202014803886414, 0.8604444265365601, 0.8671427369117737, 0.8308663368225098, 0.9312722682952881, 0.8518227934837341, 0.9253642559051514, 0.8935035467147827